<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/tobzed/Lab-1/tedwards_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: Matrix Algorithms**
**Tobias Edwards**

# **Abstract**

This first lab was on implementing the following:

1.   Inner product for vectors
2.   Matrix-vector multiplication
3.   Matrix-matrix multiplication
4.   BONUS: To implement sparse matrices with CRS and implement sparse matrix vector multiplication



#**About the code**

In [0]:
# This code is written for Lab 1 in DD2363 Methods in Scientific Computing
# Course given by the Royal Institute of Technology in Stockholm, KTH
# Code by Tobias Edwards (tedwards@kth.se), Spring 2019

# **Set up environment**

These are the environment variables, make sure to run this code before trying any code below! 

In [0]:
# Load neccessary modules.
from google.colab import files

import unittest
import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Operations on matrices and vectors are widly used in computing and numerical methods. Understanding some of the basic binary operations, such as matrix-vector mulitplication, is a fundamental requirement in order to understand more advanced methods in numerical calculation. Therefore, this lab introduces the reader to how these basic operations can be implemented in a programming language.

I have solved the problems using Python and the [numpy](http://www.numpy.org) package in order to utilize the more efficient multidimensional array. I have written unittests in order to test and verify my code. 



# **Methods**

The inner product is given by:
$\sum_{i=1}^n x_iy_i$ for two vectors $x,y \in R^n$.

The matrix vector product between a matrix $A \in R^{m\times n}$ and vector $x \in R^n$ results in a vector $y \in R^{m}$. Each element in $y$ is calculated by $\sum_{j=1}^n a_{ij}x_j$. Thus the running time for matrix multiplication in my implementation is $O(mn)$.

Each element $c_{ij}$ from the result of matrix matrix product for two matrices $A \in R^{n\times m}$ and $B \in R^{m\times p}$ is given by $\sum_{k=1}^m a_{ik}b_{kj}$. Thus the time complexity for my implementation of this method is $O(nmp)$. 

For the sparse matrix representation, Compressed Row Storage was used. This format stores three one dimensional arrays. The value array stores all non-zero elements, the column array stores which column each non-zero element has in the original matrix and the row pointer array stores indices where each index states is a pointer to which element in the value array starts a new row in the original matrix. For details, see [Sparse Matrix](https://en.wikipedia.org/wiki/Sparse_matrix).

# **Results**

Make sure the environment variables are set up.

In [0]:
# Here are the methods for the lab

def inner_product(x,y):
    if x.ndim != 1 or y.ndim != 1 or x.size != y.size: # check that the vectors dimensions are correct
        print("error in func<inner_product>: incompatible vectors")
        return
    res = 0
    for i in range(x.size):
        res += x[i] * y[i]
    return res

  
def matrix_vec_prod(A,x):
    A_shape = A.shape   # matrix A has shape (rows x cols)
    if A_shape[1] != x.size or x.ndim != 1:
        print ("error in func<matrix_vec_prod>: incorrect dimensions")
        return
    prod = np.zeros( ( A_shape[0] ) )
    for r in range(A_shape[0]):
        for c in range(A_shape[1]):
            prod[r] += A[r][c]*x[c]
    return prod

  
def matrix_matrix_prod(A,B):
    if A.shape[1] != B.shape[0]: # if A's cols are not the same in count as B's rows, then AB is undefined
        print ("error in func<matrix_matrix_prod>: incompatible matrix dimensions")
        return
    C = np.zeros((A.shape[0],B.shape[1])) # if A is (n x m) and B is (m x p), then C is (n x p)
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            for k in range(A.shape[1]):
                C[i][j] += A[i][k]*B[k][j]
    return C

  
def SparseMatrix_vec_prod(A,x):
    prod = np.zeros( (A.row_ptr.size-1) )
    for i in range(A.row_ptr.size-1):
        for j in range(A.row_ptr[i],A.row_ptr[i+1]):
            prod[i] += A.val[j]*x[A.col_idx[j]]
    return prod

  
class SparseMatrix:
    def __init__(self, val, col_idx, row_ptr):
        self.val = val
        self.col_idx = col_idx
        self.row_ptr = row_ptr

    def __str__(self): # if x = SparseMatrix, then we can call print x for a string representation of x
        mtx_str = "val array: " + np.array_str(self.val) + "\ncol_idx: " + np.array_str(self.col_idx) + "\nrow_ptr: " + np.array_str(self.row_ptr)
        return mtx_str


In [14]:

# here are the unit tests for lab 1

class TestMatrixVectorFunctions(unittest.TestCase):

    def test_inner_product(self):
        x = np.array([1,2,3,4])
        y = np.array([1,2,3,4])
        self.assertEquals(inner_product(x,y),np.inner(x,y)) #np.inner(a,b) is numpy's own inner product
        z = np.array([0,0,0,0])
        self.assertEquals(inner_product(x,z),0) # inner product with the 0-vector gives 0
        w = np.array([1,2,3])
        self.assertEquals(inner_product(x,w),None) # inner product with incompatibles vectors is undefined

    def test_matrix_vec_prod(self):
        x = np.array([1,2,3,4])
        I = np.array([  [1,0,0,0], # the identity matrix
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]
                    ])
        self.assertEquals(matrix_vec_prod(I,x).tolist(),x.tolist()) # multiplication Ix should result in the same element x
        self.assertEquals(matrix_vec_prod(I,np.array([1,2,3])), None) # multiplication where dimensions betweeen a matrix columns and a vectors rows don't match is undefined

        b = np.array([1,2,3])
        A = np.array([  [1,20,3],
                        [-2,1,4],
                        [9,-12,0],
                        [1,1,6]
                    ])
        self.assertEquals(matrix_vec_prod(A,b).tolist(),A.dot(b).tolist()) # A.dot(b) is numpy's matrix vector multiplication 
        self.assertEquals(matrix_vec_prod(A,x),None)

    def test_matrix_matrix_prod(self):
        I = np.array([  [1,0,0,0],
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]
                    ])
        A = np.array([  [1,20,3],
                        [-2,1,4],
                        [9,-12,0],
                        [1,1,6]
                    ])
        self.assertEquals(matrix_matrix_prod(A,I), None)
        B = np.array([  [1,2,3,4],
                        [0,2,4,-2],
                        [-2,9,6,1],
                        [0,4,3,2]
                    ])
        self.assertEquals(matrix_matrix_prod(B,I).tolist(),matrix_matrix_prod(I,B).tolist()) # for matrix B and indentity I: BI = IB ...
        self.assertEquals(matrix_matrix_prod(B,I).tolist(),B.tolist()) # ... and BI = B
        self.assertEquals(matrix_matrix_prod(B,A).tolist(),np.matmul(B,A).tolist()) # matmul is numpy's matrix matrixx multiplication
        self.assertEquals(matrix_matrix_prod(A,B), None) # BA is defined above but AB is undefined due to incompatible dimensions

    def test_SparseMatrix_class(self):
        val = np.array([1,2,3,4,5])
        col_idx = np.array([0,1,2,3,4])
        row_ptr = np.array([0,1,2,3,4,5])
        A = SparseMatrix(val,col_idx,row_ptr)
        self.assertEquals(A.val.tolist(),val.tolist())
        self.assertEquals(A.col_idx.tolist(),col_idx.tolist())
        self.assertEquals(A.row_ptr.tolist(),row_ptr.tolist())

    def test_SparseMatrix_vec_prod(self):
        val = np.array([4,2,3,4,5,-2,6,2])
        col_idx = np.array([0,1,1,4,1,2,3,4])
        row_ptr = np.array([0,0,2,4,4,8])
        A_sparse = SparseMatrix(val,col_idx,row_ptr)
        A = np.array([
            [0,0,0,0,0],
            [4,2,0,0,0],
            [0,3,0,0,4],
            [0,0,0,0,0],
            [0,5,-2,6,2]
        ])
        I_sparse = SparseMatrix(np.array([1,1,1,1,1]), np.array([0,1,2,3,4]), np.array([0,1,2,3,4,5]))
        I = np.array([
            [1,0,0,0,0],
            [0,1,0,0,0],
            [0,0,1,0,0],
            [0,0,0,1,0],
            [0,0,0,0,1]
        ])
        x = np.array([-2,3,7,10,1])
        print(SparseMatrix_vec_prod(A_sparse,x))
        self.assertEquals(SparseMatrix_vec_prod(A_sparse,x).tolist(),A.dot(x).tolist())
        self.assertEquals(SparseMatrix_vec_prod(I_sparse,x).tolist(),x.tolist())
        
unittest.main(argv=[''], verbosity=2, exit=False)


test_SparseMatrix_class (__main__.TestMatrixVectorFunctions) ... /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: Please use assertEqual instead.
ok
test_SparseMatrix_vec_prod (__main__.TestMatrixVectorFunctions) ... ok
test_inner_product (__main__.TestMatrixVectorFunctions) ... ok
test_matrix_matrix_prod (__main__.TestMatrixVectorFunctions) ... ok
test_matrix_vec_prod (__main__.TestMatrixVectorFunctions) ... 

[ 0. -2. 13.  0. 63.]
error in func<inner_product>: incompatible vectors
error in func<matrix_matrix_prod>: incompatible matrix dimensions
error in func<matrix_matrix_prod>: incompatible matrix dimensions
error in func<matrix_vec_prod>: incorrect dimensions
error in func<matrix_vec_prod>: incorrect dimensions


ok

----------------------------------------------------------------------
Ran 5 tests in 0.013s

OK


# **Discussion**

The lab was straight forward. I had never used numpy previously, so it took a bit of getting use to. More testing could always be recommended to catch special edge cases. I find it somewhat disconcerting that numpy doesn't really differ between row and column vectors in the same way that MatLab does. Though not a real problem, just something to get use to. It would be intersting to implement different sparse matrix models and examine how they perform against each other. 